## Импорт библиотек

In [132]:
import sqlite3
import pandas as pd

## Загрузка данных

In [133]:
genstudio = pd.read_csv('/content/genstudio.csv')
meta = pd.read_csv('/content/metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Посмотрим на данные

In [134]:
genstudio.head()

,Unnamed: 0,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022


In [135]:
genstudio['Sample ID'].value_counts()

202341831114R05C01    68516
202341831114R03C02    68516
202341831114R06C02    68516
202341831114R01C02    68516
202341831114R11C01    68516
202341831127R02C02    68516
202341831114R12C01    68516
202341831114R09C02    68516
202341831114R10C01    68516
202341831114R08C02    68516
202341831114R05C02    68516
202341831114R06C01    68516
202341831114R07C02    68516
202341831114R10C02    68516
202341831127R02C01    68516
202341831114R11C02    68516
202341831114R09C01    68516
202341831127R03C01    68516
202341831114R02C01    68516
202341831114R07C01    68516
202341831114R02C02    68516
202341831114R01C01    68516
202341831114R04C01    68516
202341831114R03C01    68516
202341831114R04C02    68516
202341831127R01C02    68516
202341831127R01C01    68516
202341831114R08C01    68516
202341831114R12C02    68516
202341831127R03C02    13036
Name: Sample ID, dtype: int64

In [136]:
genstudio.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [137]:
meta.head()

,Unnamed: 0,dna_chip_id,breed,sex
0,0,202290551164R09C01,Д,Хр
1,1,202341831114R02C01,Д,Хр
2,2,202341831114R03C01,Д,Хр
3,3,202341831114R04C01,Д,Хр
4,4,202290551140R01C01,Д,Хр


In [138]:
meta.breed.value_counts()

Д    841
Name: breed, dtype: int64

In [139]:
meta.sex.value_counts()


Хр    841
Name: sex, dtype: int64

In [140]:
meta.dna_chip_id.value_counts()

203071331121R10C01    1
203085581117R08C01    1
202290551174R07C01    1
202341831114R01C02    1
203071331128R12C02    1
                     ..
203085581141R11C02    1
203071331128R10C01    1
203010781118R06C02    1
202296001112R06C02    1
202290551164R11C01    1
Name: dna_chip_id, Length: 841, dtype: int64

In [141]:
meta.rename(columns={'Unnamed: 0': 'meta_ID'}, inplace=True)

## Создание таблиц

Напишем функцию для создания запросов, которые будут создавать таблицы с нужными колонками и primary key:

In [142]:
def create_table_query(df, table_name, key_name):
    table_names = list(df.columns)
    types = []
    for dtype in df.dtypes:
        types.append(dtype.kind)
    dict_types = {'i': 'INTEGER', 'f': 'REAL', 'O': 'TEXT'}
    table_types = [dict_types.get(n, n) for n in types]
    query = 'CREATE TABLE IF NOT EXISTS ' + table_name + ' ('
    for i in range(len(table_names)):
        query += '[' + table_names[i] + '] ' + gen_types[i]
        if table_names[i] == key_name:
            query += ' PRIMARY KEY'
        if i != len(table_names) - 1:
            query += ', '
    query += ')'
    return query

## Создание базы и таблиц

In [143]:
connection = sqlite3.connect('/content/sample_data/gen.db')

In [144]:
query_genstudio = create_table_query(genstudio, 'data', 'ID')
query_meta = create_table_query(meta, 'metadata', 'meta_ID')

In [145]:
connection.execute(query_genstudio)
connection.execute(query_meta)

Для заполнения таблиц напрямую из датафрейма воспользуемся функцией to_sql:

In [146]:
genstudio.to_sql(name='data', con=connection, if_exists='append', index=False)
meta.to_sql(name='metadata', con=connection, if_exists='append', index=False)
connection.commit()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Создадим таблицу на основе исходных таблиц, которая будет объединять значения из двух таблиц по одинаковому значению Sample ID/dna_chip_id:

In [147]:
query_ref_table = '''CREATE TABLE IF NOT EXISTS sample_link (ID INTEGER, meta_ID INTEGER, FOREIGN KEY (ID) REFERENCES data (ID), FOREIGN KEY (meta_ID) REFERENCES metadata (meta_ID))'''
connection.execute(query_ref_table)
connection.commit()

In [148]:
query_join = '''INSERT INTO sample_link SELECT d.ID, m.meta_ID FROM data d LEFT JOIN metadata m ON d.[Sample ID] = m.[dna_chip_id]'''
connection.execute(query_join)
connection.commit()

## Команда UPDATE

In [149]:
query_new_col = '''ALTER TABLE data ADD COLUMN GC_con INTEGER'''
query_update = '''UPDATE data SET [GC_con] = CASE WHEN [GC Score] = 0 THEN 0 ELSE 1 END'''
connection.execute(query_new_col)
connection.execute(query_update)
connection.commit()

## Команда DELETE

In [150]:
query_new_obs = '''INSERT INTO metadata (meta_ID, dna_chip_id, breed, sex) VALUES (841, 0, 0, 0)'''
connection.execute(query_new_obs)
connection.commit()


In [151]:
query_delete = '''DELETE FROM metadata WHERE dna_chip_id = 0'''
connection.execute(query_delete)
connection.commit()

In [152]:
connection.close()